In [ ]:
import copy
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import sklearn
from   sklearn import linear_model


# Why visualize data?  Anscombe's Quartet  

### Summary statistics like mean, standard deviation, correlation, variance explained, t-tests, F-statistics, etc., are fantastic tools.  Still,  different data sets can be fundamentally very different, but still "look the same" when viewed through these lenses.   [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet).  Gives a nice example.  Data visualization adds a very useful sanity check for important similarities/differences that summary statistics may miss.





#  Roadmap to proficiency in ANOVA

This notebook provides a concrete visual description of the type of least-squares model that underlies ANOVA.  There are lots of other **very useful** practical aspects of working with ANOVA's that you can **absorb with very little time + effort**.  

* One good way to get that experience is to look at about 8 carefully chosen examples of different outcomes. [This website](https://psychstat3.missouristate.edu/Documents/MultiBook3/Mlt08.htm) has those examples.  Just scroll to the bottom!  
* Statistical tests are an integral part of ANOVA that we won't much cover here.  Scroll to the bottom to see a few notes on this aspect of the model.

The main rule of thumb when expanding your ANOVA abilities is to find resources that work for you; google, ask friends and teachers, and check back in from time to time.  There are good resources out there, it just takes finding them.

# This lesson's math fact: mean minimizes sum of squares

Suppose you are given a cloud of points $\{p_0, \ldots, p_n \}$ and asked to find a point $q$ such that the sum of squared distances $d(q,p_1)^2 + \cdots + d(q,p_n)^2$ is as small as possible.  **Fact:** 
$$
q = \text{mean}(p_1, \ldots, p_n)
$$ 
is the unique solution.

# ANOVA

### What is it?

Linear regression where the predictor variables are all 0 or 1.

### Wait, what?

It goes like this.  Suppose you take a collection of measurements.  Each time you measure, you record come categorical independent variables, and some possibly-decimal-valued dependent variables.  The categories (say, flavor and dairy) might not have numbers associated with them, but you can "transform" a categorical column of a data matrix into a collection of numerical ones by one-hot encoding.  See, for example, the table below.

In [ ]:
D = dict(  flavor         = ['mint']*4 + ['cherry']*4,
           dairy          = (['no cream','no cream']+['cream','cream'])*2,
           cherry         = [0,0,0,0,1,1,1,1],
           mint           = [1,1,1,1,0,0,0,0],
           cream          = [0,0,1,1,0,0,1,1],
           no_cream       = [1,1,0,0,1,1,0,0],
           happiness      = 0.5*np.array([0.2, 0.4, 0.3, 0.6,0.6,0.7,1.4,1.6]) 
        )

df = pd.DataFrame(D)
display(df)

,flavor,dairy,cherry,mint,cream,no_cream,happiness
0,mint,no cream,0,1,0,1,0.10
1,mint,no cream,0,1,0,1,0.20
2,mint,cream,0,1,1,0,0.15
3,mint,cream,0,1,1,0,0.30
4,cherry,no cream,1,0,0,1,0.30
5,cherry,no cream,1,0,0,1,0.35
6,cherry,cream,1,0,1,0,0.70
7,cherry,cream,1,0,1,0,0.80


We'd like to understand the relationship between cherries, cream, and overall satisfaction with Italian sodas.  We'll do that by running a regression analysis that uses columns of the matrix to predict overall enjoyment.  That analysis is the essence of ANOVA!

In fact, this is where the term "ANOVA" come from.  It is a type of analysis based on linear regression, and linear regression is all about analyzing variance.

# Factors (flavor, dairy) vs Levels (cherry, mint)

In  ANOVA lingo,
* **factor** means "independent variable"
* **level** is a value that one of the factors takds
So, for example, flavor and dairy are factors.  Mint and cherry are levels of the flavor factor, and cream, no cream are levels of the dairy factor.


# Types of ANOVA

### Number of (in)dependent variables 

The number of (in)dependent variables has a direct bearing on how we run our regression analysis, so these names can convey some practical information about what we're dealing with.

\# independent variables  | \# dependent variables | name |
------------- | ------------- | -------------
n | * | n-way ANOVA
2+ | * | factorial ANOVA, multiple factor ANOVA, multifactor ANOVA
2+ | 1 | multiple ANOVA's
2+ | 2+ | MANOVA

### Mixed, fixed, and random effects


* Different people assign different meanings to these words.  See for example [stack exchange](https://stats..stackexchange.com/questions/4700/what-is-the-difference-between-fixed-effect-random-effect-and-mixed-effect-mode)
* So far as I can tell (?), this doesn't impact the linear regression model so mach as the statistical tools used to analyze it.  As such, we won't worry about this too much.
* The most interpretable explanation of fixed vs random vs mixed effects ANOVA I could find comes from Wikipedia: 



> Example: Teaching experiments could be performed by a college or university department to find a good introductory textbook, with each text considered a treatment. The fixed-effects model would compare a list of candidate texts. The random-effects model would determine whether important differences exist among a list of randomly selected texts. The mixed-effects model would compare the (fixed) incumbent texts to randomly selected alternatives. 


# Effects

An "effect" is a difference between to means. 


### Notation of means 

Given two independent varialbes $X$ and $Y$, let's write

\begin{align*}
\mu(X=A ) &= \text{mean value of the dependent variable(s) when $X=A$} \\
\mu(X=A, \; Y=B ) &= \text{mean value of the dependent variable(s) when $X=A$ and $Y= B$}
\end{align*}


### Main effect

A main effect is a difference between two means that only select on a single factor.  Examples:

* The main effect of cherry versus mint
$$
\mu(\text{flavor = cherry}) - \mu(\text{flavor = mint})
$$

* The main effect of cream versus no cream
$$
\mu(\text{dairy = cream}) - \mu(\text{dairy = no cream})
$$

### Simple main effect

A simple effect is a difference between two means that (i) variy the level of one factor, and (ii) hold the levels of all other factors constant.  Examples:

* The simple effect of cherry versus mint **when dairy = cream**
$$
\mu(\text{flavor = cherry, dairy = cream}) - \mu(\text{flavor = mint, dairy = cream})
$$

* The simple effect of dairy versus no dairy **when flavor = mint**
$$
\mu(\text{flavor = mint, dairy = cream}) - \mu(\text{flavor = mint, dairy = no cream})
$$


# Example: 1-way ANOVA


Let's get our feet wet with a 1-way ANOVA comparing growth rate (in centimeters per day) in seedlings to hours of sunlight received each day.



In [ ]:
S = dict(  hours  = [4,4,6,6,8,8],
           h4     = [1,1,0,0,0,0],
           h6     = [0,0,1,1,0,0],
           h8     = [0,0,0,0,1,1],
           growth =  [1.4,2.1,4.6,6.3,5.2,5.8]
        )

sf = pd.DataFrame(S)
print(sf)

   hours  h4  h6  h8  growth
0      4   1   0   0     1.4
1      4   1   0   0     2.1
2      6   0   1   0     4.6
3      6   0   1   0     6.3
4      8   0   0   1     5.2
5      8   0   0   1     5.8





### Remove one indicator column from each factor

Let's call the 0/1 columns in our table "indicator columns".  The first step in running ANOVA (or the part where we find a linear regressor, at least) is to remove one indicator column for each factor.  In the present case there is only one factor - hourse of sunlight - we we'll remove one column.  

**Which column(s) do we choose?**  Doesn't matter; at the end of the day, it will all shake out to the same thing.

**Why?** There are many answer, including (i) it is convention and (ii) we lose no information this way, since you can deduce the entries of any one indicator column, given those of the others.




In [ ]:
sfr = sf[['h6', 'h8', 'growth']]
display(sfr)

,h6,h8,growth
0,0,0,1.4
1,0,0,2.1
2,1,0,4.6
3,1,0,6.3
4,0,1,5.2
5,0,1,5.8


### Plot the data

In [ ]:
#   DEFINE A FUNCTION TO CONVERT LEVELS TO ONE-HOT VECTORS (WELL, TECHNICALLY
#   0- OR 1-HOT VECTORS)
def hours_indicators(x):
  if x == 4:
    return [0,0]
  elif x == 6:
    return [1,0]
  elif x == 8:
    return [0,1]

#   CREATE A FIGURE WITH INDICATOR COLUMNS FOR THE X AND Y COORDINATES
fig = px.scatter_3d(sfr, x = "h6", y="h8", z="growth")

#   CHANGE THE ASPECT RATIO TO CUBE
fig.update_layout(scene = dict( aspectmode = 'cube'))

#   EXPORT THE DATA TABLE TO A NUMPY ARRAY
A = sfr.to_numpy()

#   ADD DOTS FOR THE MEANS
growth_mean = dict()
for hours in [4,6,8]:
  h6,h8 = hours_indicators(hours)
  trials = sf['hours'] == hours
  growth_mean[hours] = np.mean( sf['growth'][trials])
  fig.add_trace(
      go.Scatter3d(x = [h6], y=[h8], z=[growth_mean[hours]], mode='markers', name=f'mean(hours={hours})')
  )   

#   CONNECT THE MEAN POINTS WITH DOTTED LINES
def main_effect_trace_growth(hours0, hours1):
  x0,y0 = hours_indicators(hours0)
  x1,y1 = hours_indicators(hours1)  
  trace = go.Scatter3d(  x=[x0,x1], y=[y0,y1], z = [growth_mean[hours0],growth_mean[hours1]], 
                              mode='lines', line=dict(color='black', dash='dash'),
                              name = f'MAIN EFFECT: mean(hours={hours0}) - mean(hours={hours1}) = {np.round(growth_mean[hours0]-growth_mean[hours1],4)}'
                      )
  return trace

fig.add_trace( main_effect_trace_growth(4,6) )
fig.add_trace( main_effect_trace_growth(6,8) )
fig.add_trace( main_effect_trace_growth(4,8) )
fig.show()

### Claim: The plane that passes through the three mean points is the OLS (ordinary least squares) best-fit plane for the data.

**Exercise** Use the fact about means minimizing squared distances (recorded at the top of this notebook) to convince yourself that this is true.

**Exercise** Now verify this claim numerically (hint: evaluate the regression function at points $(0,0)$, $(0,1)$, and $(1,0)$).

**Exercise** Recall that the "guess" made by the linear regression function is given by the formula 
$$
y = a_1 x_1 + a_2 x_2 + b
$$
where 
* $a_1$ is the "linear regression coefficient" on $x_1$ (in our case, $x_1$ corresponds to column `h6` of the data matrix) 
* $a_2$ is the "linear regression coefficient" on $x_2$ (in our case, $x_2$ corresponds to column `h8` of the data matrix)
* $b$ is the *intercept*.  

Extract the regression coefficients from the linear regressor, and verify that they match the main effects for 
* hours=4 verus hours=6, and
* hours=4 versus hours=8.

**Exercise** Run a 1-way ANOVA on this data (there is example code for ANOVA's at the bottom of this notebook), and verify that the quantities we've computed by hand align with those returned by python.  Note that we haven't said anything about significance testing, so don't worry about that part, for now.

# Example: 2-way ANOVA

Let's return to Italian sodas.  Recall that we have a table

In [ ]:
display(df)

,flavor,dairy,cherry,mint,cream,no_cream,happiness
0,mint,no cream,0,1,0,1,0.10
1,mint,no cream,0,1,0,1,0.20
2,mint,cream,0,1,1,0,0.15
3,mint,cream,0,1,1,0,0.30
4,cherry,no cream,1,0,0,1,0.30
5,cherry,no cream,1,0,0,1,0.35
6,cherry,cream,1,0,1,0,0.70
7,cherry,cream,1,0,1,0,0.80


### Step 1: remove "extra" columns

As in the first example, step 1 will be to remove a superflous column from each factor.

In [ ]:
dfr = df[['cherry','cream','happiness']]
display(dfr)

,cherry,cream,happiness
0,0,0,0.10
1,0,0,0.20
2,0,1,0.15
3,0,1,0.30
4,1,0,0.30
5,1,0,0.35
6,1,1,0.70
7,1,1,0.80


### Plot main effects + simple main effects

In [ ]:

#   CREATE A FICTURE WITH INDICATOR COLUMNS FOR X AND Y COORDINATES
fig = px.scatter_3d(dfr, x = "cherry", y="cream", z="happiness")
fig.update_scenes(aspectmode = 'cube')
# trace_points = go.Scatter3d(x = dfr["cherry"],y=dfr["cream"],z=dfr["happiness"], mode='markers')
# layout = go.Layout(scene = dict(xaxis_title="cherry", yaxis_title='cream', zaxis_title='happiness', aspectmode = 'cube'))
# fig = go.Figure(data = [trace_points], layout=layout)

#   EXPORT THE REDUCED TABLE TO A NUMPY ARRAY
A = dfr.to_numpy()

#   ADD DOTS FOR MEANS
happiness_simple = np.zeros((2,2))
for cherry in [0,1]:
  for cream in [0,1]:
    happiness_simple[cherry,cream] = np.mean([df['happiness'][x] for x in range(A.shape[0]) if df['cherry'][x]==cherry and df['cream'][x]==cream])
    fig.add_trace(
        go.Scatter3d(x = [cherry], y=[cream], z=[happiness_simple[cherry,cream]], mode='markers', name=f'mean(cherry={cherry}, cream={cream}) = {np.round(happiness_simple[cherry,cream],4)}')
    )     

#   ADD DOTS FOR THE MEANS
df_mean = {}
for factor in ['cherry','cream']:
  for level in [0,1]:
    indices = [x for x in range(A.shape[0]) if df[factor][x]==level]
    xyz     = dfr.iloc[indices,:].to_numpy().mean(axis=0)
    df_mean[f'{factor}{level}'] = xyz
    fig.add_trace(
        go.Scatter3d(x = [xyz[0]], y=[xyz[1]], z=[xyz[2]], mode='markers', name=f'mean({factor}={level}) = {np.round(xyz[2],4)}')
    ) 

#   ADD TRACES FOR THE SIMPLE EFFECTS
def simple_effect_trace_happiness(cherry0,cream0,cherry1,cream1):
  trace = go.Scatter3d(  x=[cherry0, cherry1], y=[cream0, cream1], z = [happiness_simple[cherry0,cream0], happiness_simple[cherry1, cream1]], 
                              mode='lines', line=dict(color='magenta', dash='dash'),
                              name = f'MAIN EFFECT (SIMPLE):mean(cherry={cherry0}, cream={cream0}) - mean(cherry={cherry1}, cream={cream1} = {np.round(happiness_simple[cherry0,cream0]-happiness_simple[cherry1,cream1],4)}'
  )
  return trace

fig.add_trace( simple_effect_trace_happiness(0,0,1,0) )
fig.add_trace( simple_effect_trace_happiness(0,0,0,1) )
fig.add_trace( simple_effect_trace_happiness(0,1,1,1) )
fig.add_trace( simple_effect_trace_happiness(1,0,1,1) )


#   ADD TRACES FOR THE MAIN EFFECTS
def main_effect_trace_happiness(factor):
  x0,y0,z0 = df_mean[f'{factor}0']
  x1,y1,z1 = df_mean[f'{factor}1']  
  trace = go.Scatter3d(  x=[x0, x1], y=[y0, y1], z = [z0, z1], 
                              mode='lines', line=dict(color='black', dash='dash'),
                              name = f'MAIN EFFECT: mean({factor}=1) - mean({factor}=0) = {np.round(z1-z0,4)}'
  )
  return trace

fig.add_trace( main_effect_trace_happiness('cherry') )
fig.add_trace( main_effect_trace_happiness('cream')  )



fig.show()




### Find the best fit plane

Something really interesting happened in the plot above.  Notice that *there is no plane* that passes through the four "simple" means (the ones at the four corners).  In paricular, if you find a plane that passes through three of these points, it won't pass through the fourth.  But there **IS** a plane the passes through the four "main" means (the ones the lie on the line segments connecting the simple ones).

This is no coincidence.  It takes a little work to prove (and if not careful the notation for the proof can get quite messy), but no matter the values we input for the dependent variable, the four "main" means will **always** lie on a single 2d plane.  Cool, right?!

This fact might seem so cool to you that you'd even venture to guess that the plane defined by these four points is the best fit plane for the whole data set.  If so, then you'd be right!

In [ ]:
#   COPY THE FIGURE
fig_fitplane = copy.deepcopy(fig)

#   EXPORT THE REDUCED TABLE TO A NUMPY ARRAY
A = dfr.to_numpy()

#   FIND A REGRESSOR TO PREDICT HAPPINESS BASED ON CHERRIES AND CREAM
# FORMAT THE REGRESSOR
reg       =   sklearn.linear_model.LinearRegression()
reg.fit(A[:,[0,1]], A[:,[2]])

# WRITE A FUNCTION TO CALCULATE Z-COORDINATES OF THE FOUR CONER POINTS OF THE BEST-FIT PLANE WE WANT TO DRAW
def axis_limits_to_corner_vectors(axis_limits):
  axis_limits = np.array(axis_limits).reshape(-1)
  row_vectors = np.array( [ [0, 1, 0, 1], 
                            [0, 0, 1, 1]]).T
  return axis_limits[row_vectors]


# CALCULATE THE Z-COORDINATES 
axis_limits = [0,1] 
corner_xy = axis_limits_to_corner_vectors(axis_limits)
z_coords = reg.predict(corner_xy).reshape(2,2)

# MAKE THE SURFACE TRACE
trace_fitplane   =          go.Surface(   
                            x=axis_limits, 
                            y=axis_limits,
                            z=list(z_coords),
                            opacity = 0.5,
                            name='best fit plane',
                            showscale=False)

print(type(trace_fitplane))

# ADD THE TRACE TO THE LIST OF TRACES
fig_fitplane.add_trace(trace_fitplane)

# ADD TITLE
fig_fitplane.update_layout(title=f'intercept = {reg.intercept_}, regression coefficients = {reg.coef_}')

# SHOW THE PLANE
fig_fitplane.show()


<class 'plotly.graph_objs.Surface'>


### Exercises

* plot the mean point of the whole data set; verify that the best fit plane passes through this point
  * note: the mean of the dependent variable, running over all rows of the data matrix, is called the **grand mean**
* verify that that the regression coefficients (ie, the $a_1$ and $a_2$ in the equation $y = a_1 x_1 + a_2 x_2$ match the main effects for flavor and dairy
* run a 2-way ANOVA on this data; find where the quantities labeled in the figure above appear in the ANOVA report

# Interaction effects

The interaction effect measures the degree to which the "simple" means fail to fall along a plane.  It has various definitions, including
* the difference in height between (i) any point $p$ in the set of four simple means and (ii) the point in $P$ that shares the same x and y coordinates with $p$, where $P$ is the plane that passes through the other three points
* the difference between the two simple effects for factor X (where X is either factor)


### Visuals

Let's explore this idea visually.  For each subset of three "simple means", we want to plot the plane that passes through those points.  There are several ways to do this: 
* Use linear regression again!  In particular, if we we ask sklearn to fit three points it will do so perfectly.  This is what we'll do.
* "Just figure it out" on a case by case basis, using what we know about the positions of the points, their relative heights, etc.  There's a ton to be gained from this approach, but it's laborious, 
* Use linear algebra; this is a very good option, too, but not everyone knows it, 

In [ ]:
#   WRITE A FUNCTION TO GENERATE TRACES FOR A PLANE AND "INTERACTION EFFECT
#   LINE" FOR EACH SUBSET OF 3 "SIMPLE" MEANS

def interaction_data(pointcloud,row_number):

  #   SELECT OUT ALL POINTS BUT ONE
  row_indices = [x for x in range(4) if not x == row_number]
  plane_points = pointcloud[row_indices, :]

  #   FIND A REGRESSOR FOR THESE THREE POINTS
  reg       =   sklearn.linear_model.LinearRegression()
  reg.fit(plane_points[:,[0,1]], plane_points[:,[2]])

  #   CALCULATE THE Z-COORDINATES NEEDED TO PLOT THE PLANE
  axis_limits = [0,1] 
  corner_xy = axis_limits_to_corner_vectors(axis_limits)
  z_coords = reg.predict(corner_xy).reshape(2,2)

  #   MAKE THE SURFACE TRACE
  trace_fitplane   =          go.Surface(   
                              x=axis_limits, 
                              y=axis_limits,
                              z=list(z_coords),
                              opacity = 0.5,
                              name='plane through 3 simple means',
                              showscale=False)
  
  #   MAKE A TRACE FOR THE LINE FROM THE LEFT-OUT-POINT TO THE PLANE-POINT THAT
  #   LIES DIRECTLY ABOVE OR BELOW IT

  x0,y0,z0 =  pointcloud[row_number]
  x1,y1    =  x0,y0
  z1       =  reg.predict(np.array([[x0,y0]]))[0,0] # <-- we have to tack [0,0] on the end because, by default, reg.predict returns a 2d array

  trace_interactionline   =   go.Scatter3d(
                              x=[x0,x1],
                              y=[y0,y1],
                              z=[z0,z1],
                              mode='lines',
                              line=dict(color='red'),
                              name=f'interaction effect: vertical distance = {np.round(np.abs(z1-z0),4)}'
                              )
  return trace_fitplane, trace_interactionline





In [ ]:
#   COPY THE FIGURE
fig_interaction = copy.deepcopy(fig)

#   PULL OUT A MATRIX WITH THE 4 "SIMPLE" MEANS AS ROWS
means_simple = []
for cherry in [0,1]:
  for cream in [0,1]:
    indices = [x for x in range(df.shape[0]) if df['cherry'][x]==cherry and df['cream'][x]==cream]
    means_simple.append(dfr.to_numpy()[indices,:].mean(axis=0))
means_simple = np.array(means_simple)
print(means_simple)



[[0.    0.    0.15 ]
 [0.    1.    0.225]
 [1.    0.    0.325]
 [1.    1.    0.75 ]]


In [ ]:
#   PLOT

#   LEAVE OUT 0
left_out_row = 0
fig_interaction = copy.deepcopy(fig)
trace_fitplane, trace_interactionline = interaction_data(means_simple, left_out_row)
fig_interaction.add_traces([trace_fitplane, trace_interactionline])
fig_interaction.show()

#   LEAVE OUT 1
left_out_row = 1
fig_interaction = copy.deepcopy(fig)
trace_fitplane, trace_interactionline = interaction_data(means_simple, left_out_row)
fig_interaction.add_traces([trace_fitplane, trace_interactionline])
fig_interaction.show()

#   LEAVE OUT 2
left_out_row = 2
fig_interaction = copy.deepcopy(fig)
trace_fitplane, trace_interactionline = interaction_data(means_simple, left_out_row)
fig_interaction.add_traces([trace_fitplane, trace_interactionline])
fig_interaction.show()

#   LEAVE OUT 3
left_out_row = 3
fig_interaction = copy.deepcopy(fig)
trace_fitplane, trace_interactionline = interaction_data(means_simple, left_out_row)
fig_interaction.add_traces([trace_fitplane, trace_interactionline])
fig_interaction.show()


### Why do all the vertical height differences come out the same?

This is an interesting question.  Here's an exercise that may help.

**Exercise** Suppose you have two independent variables, $X$ (which has levels $a$ and $b$) and $Y$ (which has levels $c$ and $d$).  Then this difference of differences:

$$
[\mu(X=a,Y=c) - \mu(X=b,Y=c)] - [\mu(X=a,Y=d) - \mu(X=b,Y=d)]
$$

equals this difference of differences:

$$
[\mu(X=a,Y=c) - \mu(X=a,Y=d)] - [\mu(X=b,Y=c) - \mu(X=b,Y=d)]
$$


### Exercise (nonessential - just for those who have time to solve puzzles)

Often times you don't need a regression solver to find the equation for a plane.  If you have time, you can try to reason out on your own why the coefficients + intercept below are the correct ones for one of the leave-one-out problems above!

In [ ]:

#   COPY THE FIGURE
fig_interaction = copy.deepcopy(fig)

#   FIND THE EQUATION FOR THE PLANE PASSING THROGH THREE OF THE SIMPLE MEANS.
#   - the equation is y = a1*x + a2*y + b
#   - claim: you can "figure out" what a1 and a2 have to be by looking at the 
#     differences in height between certain points in the plot above
#   - claim: the intercept should equal mean(cherry=0, cream=0)

intercept = 0.15  
a_x = 0.325 - intercept 
a_y = 0.225 - intercept

# CALCULATE THE Z-COORDINATES 
axis_limits = [0,1] 
corner_xy = axis_limits_to_corner_vectors(axis_limits)
z_coords = np.matmul(  np.array([[a_x,a_y]]), corner_xy.T    ).reshape(2,2) + intercept

# MAKE THE SURFACE TRACE
trace_ieplane   =          go.Surface(   
                            x=axis_limits, 
                            y=axis_limits,
                            z=list(z_coords),
                            opacity = 0.5,
                            name='best fit plane',
                            showscale=False)

print(type(trace_fitplane))

# ADD THE TRACE TO THE LIST OF TRACES
fig_interaction.add_trace(trace_ieplane)

# SHOW THE PLANE
fig_interaction.show()



<class 'plotly.graph_objs.Surface'>


# Example ANOVAs

In [ ]:
import statsmodels
from statsmodels.formula.api import ols

### Introduction to the `statsmodels` package

* [The fundamentals](https://www.statsmodels.org/devel/gettingstarted.html)
* [Simple and interaction effects](https://www.statsmodels.org/devel/examples/notebooks/generated/interactions_anova.html)

### Find the main effects

* Putting variable names inside `C( )` tells the package to treat this variable as a categorical one
  * You have to do this, because sometimes people will want to use a *single* linear regression function that takes *both* categorical *and* continuous variables as input.
* The `+` essentially means that you'll do multiple ANOVA's side by side.
  * Contrast this with the `*` below, which tells the model to look for simple and interaction effects
* **Indicator variables** for categorical factors appear in the follwing way: `C(factor)[T.level]`.  See below for illustration.

In [ ]:
formula = 'happiness ~ C(flavor) + C(dairy)'  # THE NAMES IN THIS STRING CORRESPOND TO COLUMN HEADERS IN OUR DATA FRAME
lm = ols(formula, df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:              happiness   R-squared:                       0.815
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     11.04
Date:                Tue, 14 Jul 2020   Prob (F-statistic):             0.0146
Time:                        01:18:33   Log-Likelihood:                 6.8859
No. Observations:                   8   AIC:                            -7.772
Df Residuals:                       5   BIC:                            -7.534
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.6625 

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=8



### Find the simple main effects + interaction effect

In [ ]:
formula = 'happiness ~ C(flavor) * C(dairy)'
lm = ols(formula, df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:              happiness   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     25.56
Date:                Tue, 14 Jul 2020   Prob (F-statistic):            0.00453
Time:                        01:18:35   Log-Likelihood:                 12.143
No. Observations:                   8   AIC:                            -16.29
Df Residuals:                       4   BIC:                            -15.97
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=8



#  Capstone problems


**Exercise (most time consuming)** Modify this notebook to make it easy to reproduce this analysis if we switch out `df` for some other data frame with two factors and two levels per factor.

**Exercise (less time consuming)** Modify this notebook to work for some particular data set you find or (randomly) generate.

**Exercise (least time consuming, but still very good!)**  Change the data in the data tables above: add or subtract rows so that not every level has the same number of samples, scale, delete, etc.  Watch how this changes the visuals!

# Connections to statistics

So far we've talked about ANOVA from the standpoint of linear regression.  How does this relate to the statistical sides of ANOVA?  We won't get into the details in this notebook, but here are some broad strokes.
* Splitting up variance
  * In linear regression we talk about sums and averages of squares: we use them to define things like *variance*, *variance explained*, *average squared error (aka residual variance)* 
  * Many of the basic quantities in ANOVA are also calculated by summing and averaging squares.  **Caveat lector** people use words like *variance explained* and *residual variance* in different ways depending on whether they're talking about linear regression and ANOVA, so, as always, just 2x check to make sure the version you have in mind matches the version your conversation partner has in mind.
  * For example, a key player on the staistical side is the so-called **$F$-statistic**, which is a fraction 
  $$
  \frac{MS_{explained}}{MS_{residual}}
  $$
where $MS_{explained}$ and $MS_{residual}$ are 
  * See [here](https://towardsdatascience.com/1-way-anova-from-scratch-dissecting-the-anova-table-with-a-worked-example-170f4f2e58ad) for a very illuminating worked example!

* Variation within versus variation between
  * One of the basic questions ANOVA seeks to answer is "does this group behave in the same way as that group."  A common way to go about this is, roughly speaking, to compare variation within a group to variation between groups.  
  * For illustration, take a look at the plots above, where we have different "columns" of points.  Here are two scenarios:
    * Scenario 1: Each column has a lot of points, and the points jump around a lot; concretely, the z-values for points in each column may range from close to 0 up through 100.  On the other hand, the mean height of the points in each column are very similar, within 0.001 of one another.  In this case it would be hard to say that the points in one column behave differently from those in the other columns.  They all just jump up and down in a wild fashion.  **There is high variation within each group, and low vatiation betweeen pairs of group.**.
    * Scenario 2: Each column has a lot of points, and they all stay within 0.001 of the simple mean for that column.  However, difference in mean height between any two columns is huge - in the 100's or 1000's.  **There is low variation within each group, and high vatiation betweeen pairs of group**.
  * Broadly speaking, **when variation between pairs of groups outweighs variation within individual groups, we say the groups are systematically different.**
  * The main problem with the statement above is that it is vague; what does it mean to "outweight"?  The $F$-statistic, and many other tools in statistics, give precise answers to that question.
